In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from modules import utility
import os
import pandas as pd
from medpy.metric.binary import hd, assd, dc, precision
import SimpleITK as sitk

def get_surface(img):
    a = sitk.GetImageFromArray(img)
    a = sitk.BinaryContour(a)
    a = sitk.GetArrayFromImage(a)
    return a.copy()

Using TensorFlow backend.


In [2]:
mhas = open('images.txt').readlines()
mhas = [i.replace('\n','') for i in mhas]

truths = open('truths.txt').readlines()
truths = [i.replace('\n','') for i in truths]

paths = open('paths.txt').readlines()
paths = [i.replace('\n','') for i in paths]

groups = open('groups.txt').readlines()
groups = [i.replace('\n','') for i in groups]

ct_test = open('ct_test.txt').readlines()
ct_test = [i.replace('\n','') for i in ct_test]

mr_test = open('mr_test.txt').readlines()
mr_test = [i.replace('\n','') for i in mr_test]

vtp_dir = '/home/marsdenlab/projects/DeepLofting/python/vtp/'
model_dir = '/home/marsdenlab/projects/DeepLofting/python/3d_models'

vtp_files = os.listdir(vtp_dir)

print ct_test
print vtp_files[:10]
IMAGES = ct_test+mr_test
print IMAGES
codes = ['i2ifc','i2i3d','i2i','drls','oof']
truth_files = [c for c in vtp_files if 'truth' in c]
print truth_files[:20]

test_images = []
for i in range(len(mhas)):
    if any([c in mhas[i] for c in IMAGES]):
        test_images.append((mhas[i],truths[i],paths[i],groups[i]))
        
#print test_images

['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119']
['0119_0001_i2ifc.LPA_br2_br1.vtp', '0150_0001_oof.celiac_hepatic.vtp', '0179_0001_truth.C_left.vtp', 'cabg11_all_oof.lca2.vtp', '0119_0001_i2i3d.LPA_br12.vtp', '0110_0001_i2i3d.right_iliac.vtp', '0177_0000_i2i3d.in_iliac_left.vtp', '0148_1001_i2i.renal_left.vtp', '0171_0001_i2i3d.R_incrd_b4.vtp', '0119_0001_i2i.RPA_br12_br4.vtp']
['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119', 'OSMSC0006', 'OSMSC0101', 'OSMSC0171', 'OSMSC0178', 'OSMSC0110']
['0179_0001_truth.C_left.vtp', '0179_0001_truth.R_ant.vtp', '0119_0001_truth.RPA_br8_br1.vtp', 'cabg11_all_truth.rsubcl3.vtp', '0119_0001_truth.RPA_br2.vtp', '0119_0001_truth.RPA_br9_br3.vtp', '0148_1001_truth.SMA.vtp', '0177_0000_truth.in_iliac_left.vtp', 'cabg11_all_truth.lsubcl3.vtp', '0171_0001_truth.L_incrd_b8.vtp', '0150_0001_truth.IMA.vtp', '0119_0001_truth.LPA_br4_br4.vtp', '0150_0001_tr

In [ ]:
import vtk
import SimpleITK as sitk
meta_dict = {}
for i in range(len(test_images)):
    im_file = test_images[i][0]
    print im_file
    reader = vtk.vtkMetaImageReader()
    reader.SetFileName(im_file)
    reader.Update()
    im = reader.GetOutput()

    spacing = im.GetSpacing()
    extent  = im.GetExtent()
    origin  = im.GetOrigin()
    s = extent[1::2]
    model = [c for c in IMAGES if c in im_file][0]
    model = model.replace('OSMSC','')
    meta_dict[model] = {}
    meta_dict[model]['spacing'] = spacing
    meta_dict[model]['origin'] = origin
    meta_dict[model]['extent'] = extent
    #meta_dict[model]['image'] = im
print meta_dict.keys()

/home/marsdenlab/datasets/vascular_data/OSMSC0004/OSMSC0004-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0006/OSMSC0006-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0101/OSMSC0101-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0110/OSMSC0110-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0119/OSMSC0119-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0144/OSMSC0144-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0148/OSMSC0148-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0150/OSMSC0150-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0171/OSMSC0171-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0177/OSMSC0177-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0178/OSMSC0178-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0179/OSMSC0179-cm.mha
/home/marsdenlab/datasets/vascular_data/cabg11/cabg11-image.mha
['0004', '0178', '0006', '0179', '0171', '0119', '0101', '0110', '0177', '0144', '0150', 'cabg11', '0148']


In [ ]:
model_dict = {}
df = pd.DataFrame()
SKIP=2
for truth_fn in truth_files:
    print truth_fn
    truth_pd = utility.get_pds_from_files([vtp_dir+truth_fn])[0]
    
    image = truth_fn.split('_')[0]
    grp   = truth_fn.split('.')[1]
    spacing = meta_dict[image]['spacing']
    extent  = meta_dict[image]['extent'][1::2]
    origin  = meta_dict[image]['origin']
    
    truth_np = utility.pd_to_numpy_vol(truth_pd,spacing,
                                       shape=extent,origin=origin,foreground_value=1)[::SKIP,::SKIP,::SKIP]

    if np.sum(truth_np) < 10: continue
   # truth_np = get_surface(truth_np.astype(int))
    for code in codes:
        code_fn = [c for c in vtp_files if (image in c) and (code+'.' in c) and (grp+'.' in c)]
        if len(code_fn) == 0: continue
        else:
            code_fn = code_fn[0]
            print code_fn
            
            code_pd = utility.get_pds_from_files([vtp_dir+code_fn])[0]
            
            
            code_np = utility.pd_to_numpy_vol(code_pd,spacing,shape=extent,
                                               origin=origin,foreground_value=1)[::SKIP,::SKIP,::SKIP]

            if np.sum(code_np) < 1: continue
            
            #code_np = get_surface(code_np.astype(int))
            
            print truth_np.shape, code_np.shape, np.sum(truth_np),np.sum(code_np)
            
            e_hd = hd(truth_np,code_np, spacing[0])

            #e_j = utility.jaccard3D_pd_to_itk(code_pd, truth_pd,meta_dict[image]['image'])

            e_asd = assd(truth_np,code_np, spacing[0])

            #e_dc = dc(truth_np,code_np)

            #e_ap = precision(truth_np, code_np)

            TP = np.sum(truth_np*code_np)
            FP = np.sum((1-truth_np)*code_np)
            TN = np.sum((1-truth_np)*(1-code_np))
            FN = np.sum((truth_np)*(1-code_np))
            
            d = {'image': image, "group":grp,'code':code, 'hausdorf':e_hd, 'ASSD':e_asd, "TP":TP,"FP":FP,"TN":TN,"FN":FN}
            #d = {"image":image, "group":grp, 'code':code, 'hausdorf':e_hd}
            df = df.append(d, ignore_index=True)

0179_0001_truth.C_left.vtp
0179_0001_truth.R_ant.vtp
0179_0001_i2ifc.R_ant.vtp
(133, 256, 256) (133, 256, 256) 173.0 57.0
0179_0001_i2i3d.R_ant.vtp
(133, 256, 256) (133, 256, 256) 173.0 585.0
0179_0001_i2i.R_ant.vtp
(133, 256, 256) (133, 256, 256) 173.0 223.0
0179_0001_drls.R_ant.vtp
(133, 256, 256) (133, 256, 256) 173.0 324.0
0179_0001_oof.R_ant.vtp
(133, 256, 256) (133, 256, 256) 173.0 440.0
0119_0001_truth.RPA_br8_br1.vtp
0119_0001_i2ifc.RPA_br8_br1.vtp
(241, 256, 256) (241, 256, 256) 67.0 152.0
0119_0001_i2i3d.RPA_br8_br1.vtp
(241, 256, 256) (241, 256, 256) 67.0 58.0
0119_0001_i2i.RPA_br8_br1.vtp
(241, 256, 256) (241, 256, 256) 67.0 142.0
0119_0001_drls.RPA_br8_br1.vtp
(241, 256, 256) (241, 256, 256) 67.0 338.0
0119_0001_oof.RPA_br8_br1.vtp
(241, 256, 256) (241, 256, 256) 67.0 1226.0
cabg11_all_truth.rsubcl3.vtp
cabg11_all_i2ifc.rsubcl3.vtp
(196, 256, 256) (196, 256, 256) 44.0 437.0
cabg11_all_i2i3d.rsubcl3.vtp
(196, 256, 256) (196, 256, 256) 44.0 1260.0
cabg11_all_i2i.rsubcl3.vtp


0177_0000_truth.disection.vtp
0177_0000_i2ifc.disection.vtp
(454, 256, 256) (454, 256, 256) 55997.0 5767.0
0177_0000_i2i3d.disection.vtp
(454, 256, 256) (454, 256, 256) 55997.0 2753.0
0177_0000_i2i.disection.vtp
(454, 256, 256) (454, 256, 256) 55997.0 4621.0
0177_0000_drls.disection.vtp
(454, 256, 256) (454, 256, 256) 55997.0 274.0
0177_0000_oof.disection.vtp
(454, 256, 256) (454, 256, 256) 55997.0 658.0
0119_0001_truth.LPA_br6.vtp
0119_0001_i2ifc.LPA_br6.vtp
(241, 256, 256) (241, 256, 256) 166.0 243.0
0119_0001_i2i3d.LPA_br6.vtp
(241, 256, 256) (241, 256, 256) 166.0 402.0
0119_0001_i2i.LPA_br6.vtp
(241, 256, 256) (241, 256, 256) 166.0 247.0
0119_0001_drls.LPA_br6.vtp
(241, 256, 256) (241, 256, 256) 166.0 1132.0
0119_0001_oof.LPA_br6.vtp
(241, 256, 256) (241, 256, 256) 166.0 1182.0
0171_0001_truth.L_incrd_b6.vtp
0171_0001_i2ifc.L_incrd_b6.vtp
(72, 256, 256) (72, 256, 256) 32.0 131.0
0171_0001_i2i3d.L_incrd_b6.vtp
(72, 256, 256) (72, 256, 256) 32.0 257.0
0171_0001_i2i.L_incrd_b6.vtp
(72

0148_1001_i2i.celiac_splenic.vtp
(74, 256, 256) (74, 256, 256) 556.0 1704.0
0148_1001_drls.celiac_splenic.vtp
(74, 256, 256) (74, 256, 256) 556.0 84.0
0148_1001_oof.celiac_splenic.vtp
(74, 256, 256) (74, 256, 256) 556.0 136.0
0179_0001_truth.aneurysm_1.vtp
0179_0001_i2ifc.aneurysm_1.vtp
(133, 256, 256) (133, 256, 256) 1469.0 792.0
0179_0001_i2i3d.aneurysm_1.vtp
(133, 256, 256) (133, 256, 256) 1469.0 950.0
0179_0001_i2i.aneurysm_1.vtp
(133, 256, 256) (133, 256, 256) 1469.0 800.0
0179_0001_drls.aneurysm_1.vtp
(133, 256, 256) (133, 256, 256) 1469.0 2046.0
0179_0001_oof.aneurysm_1.vtp
(133, 256, 256) (133, 256, 256) 1469.0 2597.0
0179_0001_truth.L_vtbrl.vtp
0179_0001_i2ifc.L_vtbrl.vtp
(133, 256, 256) (133, 256, 256) 1885.0 468.0
0179_0001_i2i3d.L_vtbrl.vtp
(133, 256, 256) (133, 256, 256) 1885.0 233.0
0179_0001_i2i.L_vtbrl.vtp
(133, 256, 256) (133, 256, 256) 1885.0 472.0
0179_0001_drls.L_vtbrl.vtp
(133, 256, 256) (133, 256, 256) 1885.0 1383.0
0179_0001_oof.L_vtbrl.vtp
(133, 256, 256) (133, 

In [10]:
strings = []
ct_test = [c.replace('OSMSC','') for c in ct_test]
mr_test = [c.replace('OSMSC','') for c in mr_test]
print mr_test
for CODE in ['CT','MR']:
    l = ct_test
    if CODE == 'MR': l = mr_test
    strings = []
    f = open('{}_3d_err_.txt'.format(CODE),'w')
    for c in codes:

        d = df.loc[df['code']==c]
        t = [any([H==K for H in l]) for K in d['image']]

        d = d.loc[t]

        a = d.mean()
        
        s = "{} & ${:.3f}$$ \\".format(c,a['hausdorf'])
        
        strings.append((a['hausdorf'],s))

    strings = sorted(strings,key = lambda x: x[0])
    print CODE
    for s in strings:
        print s[1]
        f.write(s[1]+'\n')
    f.close()

['0006', '0101', '0171', '0178', '0110']
CT
i2ifc & $1.069$$ \
i2i & $1.110$$ \
oof & $1.135$$ \
drls & $1.147$$ \
i2i3d & $1.294$$ \
MR
i2ifc & $0.405$$ \
i2i & $0.448$$ \
oof & $0.565$$ \
drls & $0.630$$ \
i2i3d & $0.658$$ \


In [ ]:
df.to_csv('3derrs.csv')

In [ ]:
for k in meta_dict.items():
    print k

In [ ]:
import pandas as pd
from collections import OrderedDict
def df_dice(grp):
    # sum columns    
    grp = grp[['TP', 'TN', 'FP', 'FN']].sum()
    return 2*grp['TP']/(2*grp['TP']+grp['FP']+grp['FN'])
mr_images = ['0006', '0080', '0082', '0101', '0171', '0178', '0110']
ct_images = ['0004', '0177', 'cabg11', '0179', '0144', '0148', '0150', '0119']
df = pd.read_csv('3derrs.csv')
df.loc[df.image.isin(ct_images), 'type'] = 'CT'
df.loc[df.image.isin(mr_images), 'type'] = 'MR'
dfs = OrderedDict()

dfs['DICE'] = pd.concat([df.groupby(['code', 'type']).apply(df_dice).rename(columns={0:'DICE'}).unstack(),
                         pd.DataFrame(df.groupby(['code']).apply(df_dice)).rename(columns={0:'Overall'})],
                        axis=1)

for k in ['ASSD', 'hausdorf']:
    dfs[k] = pd.concat([df.groupby(['code', 'type']).mean()[k].unstack(),
                        pd.DataFrame(df.groupby(['code']).mean()[k]).rename(columns={k: 'Overall'})], axis=1)
    out_df = pd.concat(dfs.values(), keys=dfs.keys(), axis=1).sort_values(('DICE', 'Overall'), 
                                                                          ascending=True)
    out_df